In [4]:
#Part 1 a.
import tensorflow as tf
import numpy as np

# Used AI; see [1]
dataset = tf.keras.utils.image_dataset_from_directory(
    "Project2",             
    labels="inferred",      
    label_mode="int",       
    color_mode="rgb",
    image_size=(128, 128),  
    batch_size=2,           
    shuffle=True,
    seed=1
)

for batch_images, batch_labels in dataset.take(1):
    print("1 batch images shape:", batch_images.shape)   # 128, 128, 3
    print("1 batch labels shape:", batch_labels.shape)

class_names = dataset.class_names
print("Class order:", class_names)
X_batches = []
y_batches = []

for images, labels in dataset:
    X_batches.append(images.numpy())    
    y_batches.append(labels.numpy())    

X = np.concatenate(X_batches, axis=0)   
y = np.concatenate(y_batches, axis=0)   

print("Final X shape:", X.shape)
print("Final y shape:", y.shape)
print("Label values and counts:", np.unique(y, return_counts=True))


Found 21322 files belonging to 2 classes.
1 batch images shape: (2, 128, 128, 3)
1 batch labels shape: (2,)
Class order: ['damage', 'no_damage']
Final X shape: (21322, 128, 128, 3)
Final y shape: (21322,)
Label values and counts: (array([0, 1], dtype=int32), array([14170,  7152]))


In [5]:
#Task 1 b.
import numpy as np

unique, counts = np.unique(y, return_counts=True)
for label, count in zip(unique, counts):
    print(f"Label {label} ({class_names[label]}): {count} images")

# Used AI; see [2]
print("Total images:", len(y))
print("X ndim:", X.ndim)
print("X shape:", X.shape)        
print("Image height:", X.shape[1])
print("Image width:", X.shape[2])
print("Num channels:", X.shape[3])
print("Data type:", X.dtype)
print("Min pixel value:", X.min())
print("Max pixel value:", X.max())
print("Mean pixel value:", X.mean())
print("Std pixel value:", X.std())

Label 0 (damage): 14170 images
Label 1 (no_damage): 7152 images
Total images: 21322
X ndim: 4
X shape: (21322, 128, 128, 3)
Image height: 128
Image width: 128
Num channels: 3
Data type: float32
Min pixel value: 0.0
Max pixel value: 255.0
Mean pixel value: 86.06063
Std pixel value: 35.43903


In [9]:
#Task 1 c.

from sklearn.model_selection import train_test_split
import numpy as np
X_norm = X / 255.0   

print("After normalization:")
print("Min:", X_norm.min(), "Max:", X_norm.max())

X_train, X_temp, y_train, y_temp = train_test_split(
    X_norm,
    y,
    test_size=0.3,        
    stratify=y,
    random_state=1
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,        
    stratify=y_temp,
    random_state=1
)

print("Train shape:", X_train.shape, y_train.shape)

input_shape = X_train.shape[1:] 

print("Val shape:  ", X_val.shape,   y_val.shape)
print("Test shape: ", X_test.shape,  y_test.shape)

After normalization:
Min: 0.0 Max: 1.0
Train shape: (14925, 128, 128, 3) (14925,)
Val shape:   (3198, 128, 128, 3) (3198,)
Test shape:  (3199, 128, 128, 3) (3199,)


In [7]:
# Part 2 - ANN

import tensorflow as tf
from tensorflow.keras import layers, models

# Used AI; see [3]
X_train_ann = tf.image.resize(X_train, (64, 64)).numpy()
X_val_ann   = tf.image.resize(X_val,   (64, 64)).numpy()
X_test_ann  = tf.image.resize(X_test,  (64, 64)).numpy()

def build_small_dense_ann(input_shape):
    model = models.Sequential([
        layers.Flatten(input_shape=input_shape),
        layers.Dense(64, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

small_dense_model = build_small_dense_ann((64, 64, 3))
small_dense_model.summary()

history_small_dense = small_dense_model.fit(
    X_train_ann, y_train,
    validation_data=(X_val_ann, y_val),
    epochs=20,
    batch_size=64,
    verbose=1
)

small_dense_test_loss, small_dense_test_acc = small_dense_model.evaluate(
    X_test_ann, y_test, verbose=0
)
print("Small Dense ANN - Test Accuracy:", small_dense_test_acc)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 12288)             0         
                                                                 
 dense (Dense)               (None, 64)                786496    
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 786561 (3.00 MB)
Trainable params: 786561 (3.00 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
234/234 [==============================] - 2s 7ms/step - loss: 0.6691 - accuracy: 0.6603 - val_loss: 0.7772 - val_accuracy: 0.6667
Epoch 2/20
234/234 [==============================] - 2s 6ms/step - loss: 0.5846 - accuracy: 0.7079 - val_loss: 0.5439 - val_accuracy: 0.7380


In [11]:
# Part 2 - LeNet - based off of paper
def build_lenet(input_shape):
    model = models.Sequential([
        layers.Conv2D(6, (5, 5), activation="relu", padding="same",
                      input_shape=input_shape),
        layers.AveragePooling2D(pool_size=(2, 2)),
        layers.Conv2D(16, (5, 5), activation="relu"),
        layers.AveragePooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(120, activation="relu"),
        layers.Dense(84, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

lenet_model = build_lenet(input_shape)
lenet_model.summary()

history_lenet = lenet_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=64,
    verbose=1
)

lenet_test_loss, lenet_test_acc = lenet_model.evaluate(X_test, y_test, verbose=0)
print("LeNet-5 - Test Accuracy:", lenet_test_acc)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 128, 128, 6)       456       
                                                                 
 average_pooling2d_2 (Avera  (None, 64, 64, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_3 (Conv2D)           (None, 60, 60, 16)        2416      
                                                                 
 average_pooling2d_3 (Avera  (None, 30, 30, 16)        0         
 gePooling2D)                                                    
                                                                 
 flatten_2 (Flatten)         (None, 14400)             0         
                                                                 
 dense_5 (Dense)             (None, 120)              

In [15]:
def build_alt_lenet(input_shape):
    model = models.Sequential([
        # Block 1
        layers.Conv2D(32, (3, 3), activation="relu", padding="valid",
                      input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),

        # Block 2
        layers.Conv2D(64, (3, 3), activation="relu", padding="valid"),
        layers.MaxPooling2D(pool_size=(2, 2)),

        # Block 3
        layers.Conv2D(128, (3, 3), activation="relu", padding="valid"),
        layers.MaxPooling2D(pool_size=(2, 2)),

        # Block 4
        layers.Conv2D(128, (3, 3), activation="relu", padding="valid"),
        layers.MaxPooling2D(pool_size=(2, 2)),

        # Classification head
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

alt_lenet_model = build_alt_lenet(input_shape)
alt_lenet_model.summary()

history_alt_lenet = alt_lenet_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=64,
    verbose=1
)

alt_test_loss, alt_test_acc = alt_lenet_model.evaluate(X_test, y_test, verbose=0)
print("Alternate-LeNet-5 - Test Accuracy:", alt_test_acc)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 14, 14, 128)      

In [16]:
print("Dense ANN      Test Acc:", small_dense_test_acc)
print("LeNet-5        Test Acc:", lenet_test_acc)
print("Alt-LeNet-5    Test Acc:", alt_test_acc)

Dense ANN      Test Acc: 0.7127227187156677
LeNet-5        Test Acc: 0.931228518486023
Alt-LeNet-5    Test Acc: 0.9665520191192627


In [17]:
alt_lenet_model.save("damage.keras")
print("Saved best model to damage.keras!!!")

Saved best model to damage.keras!!!
